## Introduction to the Transformers Library

HuggingFace 🤗 is an AI company that has grown fast in the last few years, especially in the realm of Natural Language Processing (NLP).

In particular, the Transformers library has revolutionized the way people  work with large-scale transformer models. The goal of this exercise is to introduce you to these models for the first time and show how easy they can be to work with.

### Why you should love HuggingFace:

#### Pre-trained Models 📚:

One of the best features of the Transformers library is its huge repo of pre-trained models. Whether you're looking to employ BERT, GPT-2, T5, RoBERTa, or any of the other transformer architectures, chances are you'll find a version that suits your needs in their model hub.

#### It's super easy 👍:

The library is designed to be user-friendly. Loading a model and its corresponding tokenizer can be done in just a couple of lines of code. This simplicity extends to fine-tuning as well, allowing you to adapt these powerful models to a wide range of tasks. The `pipelines` library we'll be using lets you go from model selection to getting results in just a few lines.

#### Tokenizer  🔄 and Datasets 📊 Library:

Alongside the Transformers library, HuggingFace also offers the Tokenizers and Datasets libraries. While the first provides efficient and easy-to-use tokenization methods, the second offers a whole bunch of datasets, meaning you have all the tools and data you need in one ecosystem.

#### Community-Driven 🌐:
The HuggingFace community is very active and any community member (you included) can upload their own models and datasets.

### What You'll Learn
- How to use HuggingFace pipelines for sentiment analysis, zero-shot classification, etc.
- How to load and use pre-trained models.
- How to perform inference on text and image data.


# Environment Setup

Before using Hugging Face Transformers in this Colab notebook, we need to install a few libraries and optionally log into Hugging Face to access private models or upload content.


In [1]:
# Install required packages
!pip install -q transformers datasets sentencepiece sacremoses huggingface_hub


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 10.9 MB/s eta 0:00:00


# Optional: Hugging Face Login

If you want to use private models or push your own models/datasets to the Hugging Face Hub, log in below.  
Otherwise, you can skip this step — most public models and pipelines will work without it.


In [2]:
from huggingface_hub import login

# ⬇️ You'll be prompted to paste your Hugging Face token here
# Get it from https://huggingface.co/settings/tokens
login()


# Use GPU if Available

Transformers run faster on GPU. This cell will detect if a GPU is available and print which device will be used.


In [3]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)


Using device: cpu


## Detect Sentiment in Text

We’ll begin with a quick demo of HuggingFace’s built-in `pipeline` for sentiment analysis. This is a high-level API that allows you to run powerful transformer models with just a few lines of code.

By default, HuggingFace uses the model:
- `distilbert-base-uncased-finetuned-sst-2-english`  
Trained on movie reviews, it returns binary sentiment: `POSITIVE` or `NEGATIVE`.

Let’s see how it works!

**Note.** Over the course of this notebook, you'll be using Pipelines to download and easily use some very powerful models. Bear in mind that some of these models are quite large (up to 500Mb so make sure you have some disk space free on your machine or run this notebook in a Colab with faster download speeds!).

We are going to be using pre-built models and the best resource for implementing them will be using the [Pipelines documentation](https://huggingface.co/docs/transformers/main_classes/pipelines). If you ever want to delete the models locally after use, you can find them here in your root directory at:

`/.cache/huggingface/hub`

In [4]:
from transformers import pipeline

# Load the sentiment analysis pipeline with the default model
sentiment_pipeline = pipeline("sentiment-analysis")

# Try a very positive sentence
result = sentiment_pipeline("Transformers are awesome!")
print(result)  # [{'label': 'POSITIVE', 'score': 0.9998}]


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Device set to use cpu


[{'label': 'POSITIVE', 'score': 0.9998667240142822}]


This returns a dictionary with the predicted sentiment label and its confidence score.
For example: `{'label': 'POSITIVE', 'score': 0.999}`.


### 🧾 Output explanation:
The result is a dictionary with:
- `'label'`: POSITIVE or NEGATIVE
- `'score'`: Confidence level (between 0 and 1)


## Nuanced Sentiment/Slang Mismatch

While the default Hugging Face sentiment model works well for formal text (like movie reviews), it struggles with:
- Informal language
- Modern slang
- Short or sarcastic expressions

Why? It’s trained on the **Stanford Sentiment Treebank**, a dataset mostly based on formal movie review sentences.


### Example: Slang Mismatch

Let’s test a sentence that means something **positive** in modern slang, but may be misclassified.


In [5]:
# Try a modern slang sentence

# This may give incorrect results with the default model
print(sentiment_pipeline("These beats are sick!"))


[{'label': 'NEGATIVE', 'score': 0.9997040629386902}]



That's clearly wrong — in context, this means "These beats are great!"  
The model is missing cultural nuance.

---

### Use a Better-Fit Model: `twitter-roberta-base-sentiment`

```markdown
Let’s use a model trained on Twitter data. It's better at:
- Informal expressions
- Emojis
- Short sentences
- Modern slang


In [6]:
# Use a Twitter-optimized model

twitter_pipeline = pipeline(
    "sentiment-analysis",
    model="cardiffnlp/twitter-roberta-base-sentiment-latest"
)

print(twitter_pipeline("These beats are sick!"))
print(twitter_pipeline("These jokes were absolutely killer!"))


config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cpu


[{'label': 'positive', 'score': 0.788418173789978}]
[{'label': 'positive', 'score': 0.7827622890472412}]


You should see a much more accurate interpretation of the sentiment we're trying to express.

# Task
Try testing with:

- Emojis: "That movie was 🔥🔥🔥"
- Irony: "Just perfect... got rained on all day 🙄"
- Other expressions: "She totally crushed that pitch!"
- Compare both models. See how their predictions differ.

## Multilingual Sentiment Analysis

Many Hugging Face models are trained to understand text in multiple languages.

Let’s try out a few that support **multilingual input**, meaning you can feed in text in French, German, Spanish, etc.


**Option 1**: nlptown/bert-base-multilingual-uncased-sentiment

- Supports several European languages.
- Predicts sentiment on a 1–5 star scale.

In [7]:
# Load multilingual sentiment pipeline (1 to 5 stars)
multilang_pipeline = pipeline(
    "sentiment-analysis",
    model="nlptown/bert-base-multilingual-uncased-sentiment"
)

# Try texts in different languages
print(multilang_pipeline("Ce film était absolument incroyable !"))  # French
print(multilang_pipeline("Das war ein schrecklicher Tag."))        # German
print(multilang_pipeline("Este producto es excelente."))           # Spanish


config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu


[{'label': '5 stars', 'score': 0.7454084753990173}]
[{'label': '1 star', 'score': 0.8029126524925232}]
[{'label': '5 stars', 'score': 0.723439633846283}]


The model gives sentiment on a 1–5 star scale (not binary). You can treat:
- 1-2 stars as negative
- 3 stars as neutral
- 4-5 stars as positive

Confidence scores around 0.70–0.80 are common in informal evaluations.

#Task
Try feeding in:

- Positive and negative reviews in your native language.
- Sentences with mixed sentiment (e.g., “The service was bad but the food was great.”).
- Texts with cultural idioms and humor.

## Zero-Shot Classification

Zero-shot classification allows you to categorize text into labels **without training on those categories**.

This is made possible by **entailment models** (e.g., `facebook/bart-large-mnli`), which treat the task as a Natural Language Inference (NLI) problem.

Great for:
- Prototyping new classification tasks
- Multilingual and open-domain applications
- Multi-label classification


In [8]:
# Load the zero-shot classifier
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Multi-label input (note: works best with short lists and concise input)
output = classifier(
    "He invested in green energy and launched a startup.",
    candidate_labels=["business", "environment", "technology", "sports"],
    multi_label=True  # allow multiple labels to be true
)

# Display nicely
for label, score in zip(output["labels"], output["scores"]):
    print(f"{label}: {score:.4f}")


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


business: 0.9908
technology: 0.9725
environment: 0.9461
sports: 0.0001


### Output Explanation

- `labels`: the candidate categories sorted by confidence
- `scores`: how likely each label is, according to the model

## Grammar Correction with Transformers

We'll compare two models for grammar correction:

1. `prithivida/grammar_error_correcter_v1` — simple and fast, tailored for grammar correction
2. `vennify/t5-base-grammar-correction` — a T5 model that uses natural-language task prompts

Both work well for sentence-level fixes.


 Option 1: prithivida/grammar_error_correcter_v1

In [9]:
# Load grammar correction pipeline
corrector = pipeline("text2text-generation", model="prithivida/grammar_error_correcter_v1")

# Sentence with errors
text = "She go to school every day and play piano at evening."

# Generate correction
output = corrector(text, max_length=128)[0]['generated_text']

# Display
print("Original: ", text)
print("Corrected:", output)


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu
Both `max_new_tokens` (=256) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Original:  She go to school every day and play piano at evening.
Corrected: She goes to school every day and plays piano in the evenings.


🧪 Option 2: T5-based model with prompt — vennify/t5-base-grammar-correction

In [10]:
# Load T5 grammar correction pipeline
t5_corrector = pipeline("text2text-generation", model="vennify/t5-base-grammar-correction")

# Sentence with prompt
text = "He go to market yesterday and buy many fruit."

# Prefix the task with "fix grammar:"
output = t5_corrector("fix grammar: " + text, max_length=128)[0]['generated_text']

# Display
print("Original: ", text)
print("Corrected:", output)


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu
Both `max_new_tokens` (=256) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Original:  He go to market yesterday and buy many fruit.
Corrected: He went to market yesterday and bought many fruits.


| Feature                   | `prithivida/grammar_error_correcter_v1` | `vennify/t5-base-grammar-correction`  |
| ------------------------- | --------------------------------------- | ------------------------------------- |
| Model Type                | Custom Transformer                      | T5 (text-to-text, prompt-based)       |
| Simplicity (Beginner Use) | ✅ Easy                                  | ⚠️ Requires prompt prefix             |
| Speed                     | ⚡ Fast                                  | Slightly slower                       |
| Versatility               | ❌ Only grammar correction               | ✅ Multi-task capable                  |
| Best for                  | Quick grammar correction demos          | Teaching task prompting & general NLP |


##Task: Try It Yourself
- "They is going to the school in Mondays."
- "I not go to the store yesterday."
- "He has too much books for his bag."

## Text Summarization

Transformer-based models can extract or generate **summaries** of long documents. This works best with encoder-decoder models like **BART** and **T5**.

We’ll use Hugging Face’s `pipeline("summarization")` with the model:
- `facebook/bart-large-cnn` (fine-tuned for summarizing news articles)


In [11]:
# Load summarization pipeline
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# Long input text
text = """
Artificial intelligence (AI) in education refers to the application of machine learning algorithms and other AI technologies
to enhance teaching and learning processes. AI-powered platforms can adapt instructional content to meet individual student
needs, provide real-time feedback, and support educators in identifying learning gaps. In recent years, the integration of AI
in classrooms has increased significantly, driven by advances in natural language processing and data analytics. Researchers
continue to explore the long-term impact of these technologies on student engagement, educational outcomes, and pedagogical practices.
"""

# Generate summary
summary = summarizer(text, max_length=60, min_length=20, do_sample=False)[0]['summary_text']
print("Summary:", summary)


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


Summary: Artificial intelligence (AI) in education refers to the application of machine learning algorithms and other AI technologies to enhance teaching and learning processes. AI-powered platforms can adapt instructional content to meet individual student needs.


**!Notes**

- Summarization models are abstractive — they generate new sentences, not just extract.
- You can control output with min_length, max_length, and do_sample.

## Named Entity Recognition (NER)

NER is the task of identifying **entities** (like persons, organizations, and locations) in text and labeling them with their category.

We’ll use Hugging Face's pipeline with a BERT model fine-tuned on the CoNLL-2003 dataset:
- `dslim/bert-base-NER`


In [12]:
# Load the NER pipeline
ner_pipeline = pipeline("ner", grouped_entities=True, model="dslim/bert-base-NER")

# Sample text
text = """
Ada Lovelace is widely considered one of the first computer programmers. She collaborated with Charles Babbage on his proposed mechanical general-purpose computer, the Analytical Engine. Lovelace studied mathematics in London and her contributions are
recognized by institutions such as the University of Oxford and the Association for Computing Machinery.
"""

# Run NER
entities = ner_pipeline(text)

# Display results
for entity in entities:
    print(f"{entity['word']}: {entity['entity_group']} ({entity['score']:.2f})")


config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu
/usr/local/lib/python3.12/dist-packages/transformers/pipelines/token_classification.py:186: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.SIMPLE"` instead.
  warnings.warn(


Ada Lovelace: PER (1.00)
Charles Babbage: PER (0.99)
Ana: ORG (0.53)
Engine: MISC (0.51)
Lovelace: PER (0.95)
London: LOC (1.00)
University of Oxford: ORG (1.00)
Association for Computing Machinery: ORG (1.00)


Expected output:

Ada Lovelace: PER (1.00)

Charles Babbage: PER (0.99)

Ana: ORG (0.53)

Engine: MISC (0.51)

Lovelace: PER (0.95)

London: LOC (1.00)

University of Oxford: ORG (1.00)

Association for Computing Machinery: ORG (1.00)

### Interpreting the Output

The model did a great job identifying well-known entities:

| Entity                                | Label | Confidence | Notes                          |
|---------------------------------------|-------|------------|--------------------------------|
| Ada Lovelace                          | PER   | 1.00       | ✔️ Correct identification       |
| Charles Babbage                       | PER   | 0.99       | ✔️ Person entity                |
| London                                | LOC   | 1.00       | ✔️ Location                     |
| University of Oxford                  | ORG   | 1.00       | ✔️ Academic institution         |
| Association for Computing Machinery   | ORG   | 1.00       | ✔️ Recognized organization      |

However, a few **quirks** appear in the output:

| Extracted Entity | Label | Issue |
|------------------|-------|-------|
| Ana              | ORG   | ❌ Likely a tokenization error from "Analytical" |
| Engine           | MISC  | ⚠️ May not be useful in this context |
| Lovelace         | PER   | ⚠️ Detected again as a standalone name |

### 💡 What to Learn

- The model works well with well-formed, factual text.
- Tokenization can split unexpected chunks (e.g., "Ana" from "Analytical").
- Some names/entities may be repeated with slightly different confidence levels.
- In production, you may want to **post-process results** (e.g., merge repeated entities, filter out low-confidence tags).



## Test Alternative NER Model

In [13]:
# Load a more conservative multilingual NER model
ner_pipeline = pipeline(
    "ner",
    model="Davlan/bert-base-multilingual-cased-ner-hrl",
    grouped_entities=True
)

entities = ner_pipeline(text)
for e in entities:
    print(f"{e['word']}: {e['entity_group']} ({e['score']:.2f})")


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/709M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/264 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu


Ada Lovelace: PER (1.00)
Charles Babbage: PER (1.00)
Lovelace: PER (1.00)
London: LOC (1.00)
University of Oxford: ORG (1.00)
Association for Computing Machinery: ORG (1.00)


### ✅ Using an Alternative NER Model to Improve Results

The model `Davlan/bert-base-multilingual-cased-ner-hrl` provides better handling of multi-word entities and avoids some tokenization artifacts.

This version:
- Removed incorrect entities like `"Ana"` and `"Engine"`
- Correctly grouped `"Association for Computing Machinery"`
- Still repeated `"Lovelace"`, which can be cleaned up in post-processing

This demonstrates a key NLP lesson:
> 📌 **Model selection matters.** Trying multiple pre-trained models can significantly improve performance in downstream tasks.


### Bonus Task:
Try a different domain:
- `"Tesla was founded by Elon Musk in California and is now headquartered in Texas."`
- `"Marie Curie conducted pioneering research on radioactivity at the Sorbonne."`

Which model handles names, locations, and organizations best in your example?


## Machine Translation

Hugging Face offers many pre-trained models for translating between languages.  
We’ll use the `translation` pipeline with a model from the **Helsinki-NLP/opus-mt** family.

These models are trained on millions of bilingual sentence pairs and support dozens of language pairs.


In [14]:
# English to German
translator_en_de = pipeline("translation_en_to_de", model="Helsinki-NLP/opus-mt-en-de")
print(translator_en_de("The weather today is sunny with a light breeze.")[0]['translation_text'])

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/298M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/768k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/298M [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/797k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

Device set to use cpu


Das Wetter heute ist sonnig mit einer leichten Brise.


In [15]:
# French to English
translator_fr_en = pipeline("translation", model="Helsinki-NLP/opus-mt-fr-en")
print(translator_fr_en("Les modèles de langage sont formés sur de grandes quantités de texte.")[0]['translation_text'])


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/301M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

Device set to use cpu


Language models are formed on large amounts of text.


## ❓ Question Answering

Question answering (QA) models take a **context paragraph** and a **question**, and return a **text span** from the context that answers the question.

We'll use the `pipeline("question-answering")` with the model:
- `distilbert-base-cased-distilled-squad`


In [16]:
# Load QA pipeline
qa_pipeline = pipeline("question-answering", model="distilbert-base-cased-distilled-squad")

# Define context and question
context = """
The Large Hadron Collider (LHC) is the world's largest and most powerful particle accelerator.
It was built by the European Organization for Nuclear Research (CERN) and lies in a tunnel 27 kilometers in circumference
beneath the France–Switzerland border near Geneva. The LHC first started operations in 2008 and has since been used to
discover the Higgs boson particle.
"""

question = "Where is the Large Hadron Collider located?"

# Run QA
answer = qa_pipeline(question=question, context=context)

print("Question:", question)
print("Answer:", answer['answer'])


config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Device set to use cpu


Question: Where is the Large Hadron Collider located?
Answer: Geneva


**!Notes**
- QA models are fine-tuned on datasets like SQuAD.
- They do not generate new answers — they extract spans from the context.
- The quality of the answer depends heavily on how closely the question aligns with the context.




## Summary

You've now seen how to apply Hugging Face Transformers to a wide variety of NLP tasks:

- Sentiment Analysis
- Nuanced & Multilingual Sentiment
- Zero-Shot Classification
- Grammar Correction
- Summarization
- Named Entity Recognition
- Translation
- Question Answering

### 📚 Recommended Resources

- 🤗 [Hugging Face Course](https://huggingface.co/course) – free, beginner-friendly, and practical
- 📖 [Transformers Docs](https://huggingface.co/docs/transformers) – for deep dives and advanced usage
- 💬 [Hugging Face Forums](https://discuss.huggingface.co/) – great for getting help and connecting with the community
- 🧪 [Papers with Code](https://paperswithcode.com/) – see state-of-the-art models by task


